# 因果语言模型训练实例

## Step1 导入相关包

In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, BloomForCausalLM

C:\Users\Administrator\.conda\envs\py310torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("./wiki_cn_filtered/")

In [3]:
ds

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
})

In [4]:
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

## Step3 数据集处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-389m-zh")

def process_func(examples):
    contents = [e + tokenizer.eos_token for e in examples["completion"]]
    return tokenizer(contents, max_length=384, truncation=True)

In [6]:
tokenized_ds = ds.map(process_func, batched=True, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 10000
})

In [7]:
from torch.utils.data import DataLoader

dl = DataLoader(tokenized_ds, batch_size=2, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=False))

In [8]:
next(enumerate(dl))

(0,
 {'input_ids': tensor([[    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3

In [9]:
tokenizer.pad_token, tokenizer.pad_token_id

('<pad>', 3)

In [10]:
tokenizer.eos_token, tokenizer.eos_token_id

('</s>', 2)

## Step4 创建模型

In [11]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-389m-zh")

## Step5 配置训练参数

In [12]:
args = TrainingArguments(
    output_dir="./causal_lm",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    logging_steps=10,
    num_train_epochs=1,
    fp16=True
)

## Step6 创建训练器

In [13]:
trainer = Trainer(
    args=args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_18404\1098143094.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Step7 模型训练

In [14]:
trainer.train()

Step,Training Loss
10,3.975000
20,3.962700
30,3.965300
40,3.875800
50,3.882800
60,3.835700
70,3.822800
80,4.080500
90,4.406900
100,5.833000


TrainOutput(global_step=312, training_loss=7.947202181204771, metrics={'train_runtime': 643.6777, 'train_samples_per_second': 15.536, 'train_steps_per_second': 0.485, 'total_flos': 6092709626314752.0, 'train_loss': 7.947202181204771, 'epoch': 0.9984})

## Step8 模型推理

In [15]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [16]:
pipe("西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安", max_length=128, do_sample=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': '西安交通大学博物馆（Xi\'an Jiaotong University Museum）是一座位于西安 -/：“（，,((（（（（--"\n。。----/)\n".，）「大， //。“。为, ******. （333'}]

In [17]:
pipe("下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常", max_length=128, do_sample=True)

[{'generated_text': '下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常112、 (P））））与****45””），, -,；622222）是-**'}]